# Introduction
10거래일 내 단기과열 지정예고가 뜬 종목은 손실이 나는 것 같다.  
따라서 데이터를 수집하고 머신러닝의 feature로 넣어주려고 한다.  
유튜브: https://youtu.be/XKb1kOE9J2M

# Import

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import pandas as pd

# Scraping

In [2]:
stock_code = '001250'
list_1 = []
list_2 = []

driver = webdriver.Chrome()
url = 'https://kind.krx.co.kr/main.do?method=loadInitPage&scrnmode=1'
driver.get(url)

driver.find_element_by_css_selector('#AKCKwdTop').send_keys(stock_code)
driver.find_element_by_css_selector('#AKCKwdTop').send_keys(Keys.ENTER)
time.sleep(3)

driver.find_element_by_css_selector('body > section.wrapper-contents > section > article > nav > ul > li:nth-child(3) > a').click()
time.sleep(3)

source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')

dates = soup.find_all('td', class_='first')
titles = soup.find_all(id='disc')

for date in dates:
    list_1.append(date.text)
    
for title in titles:
    list_2.append(title['title'])
    
driver.quit()

In [3]:
df = pd.DataFrame(list(zip(list_1, list_2)), columns=['일자', '제목'])
df.head()

,일자,제목
0,2022-04-22 16:14,단기과열종목(3거래일 단일가매매) 지정
1,2022-04-13 16:05,[투자주의]스팸관여과다종목
2,2022-04-12 16:24,(예고)단기과열종목(3거래일 단일가매매) 지정예고
3,2022-03-30 16:30,[투자주의]스팸관여과다종목


# Date와 Time 칼럼 만들기

In [4]:
df['Date'] = pd.to_datetime(df['일자']).apply(lambda x : x.date())
df['Time'] = pd.to_datetime(df['일자']).apply(lambda x : x.time())
df.head()

,일자,제목,Date,Time
0,2022-04-22 16:14,단기과열종목(3거래일 단일가매매) 지정,2022-04-22,16:14:00
1,2022-04-13 16:05,[투자주의]스팸관여과다종목,2022-04-13,16:05:00
2,2022-04-12 16:24,(예고)단기과열종목(3거래일 단일가매매) 지정예고,2022-04-12,16:24:00
3,2022-03-30 16:30,[투자주의]스팸관여과다종목,2022-03-30,16:30:00


# csv 파일로 저장하기
파일명은 종목명이 아닌 종목코드로 해야 혼란이 없다.  
종목코드는 unique한 값이기 때문이다.

In [12]:
df = df[['Date','Time','제목']]
df.head()

,Date,Time,제목
0,2022-04-22,16:14:00,단기과열종목(3거래일 단일가매매) 지정
1,2022-04-13,16:05:00,[투자주의]스팸관여과다종목
2,2022-04-12,16:24:00,(예고)단기과열종목(3거래일 단일가매매) 지정예고
3,2022-03-30,16:30:00,[투자주의]스팸관여과다종목


In [18]:
file_name = str(df['Date'].loc[0]) + '_' + stock_code + '.csv'

In [19]:
df.to_csv(file_name, index = False)